In [1]:
import os
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
from tqdm import tqdm

In [2]:
model_name = 'LSTM'
epochs = 1000
batch_size = 32
learning_rate = 0.0001
model_path = 'model'
only_save = False
patience = 50
verbose = True
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

In [4]:
# PyTorch Dataset 類
class StockDataset(Dataset):
    def __init__(self, input_folder, target_folder, stock_list, year_list):
        self.data = []
        self.targets = []

        for stock in stock_list:
            stock_input_folder = os.path.join(input_folder, stock)
            stock_target_folder = os.path.join(target_folder, stock)

            if not os.path.exists(stock_input_folder):
                print(f"Stock folder {stock} does not exist in input.")
                continue

            if not os.path.exists(stock_target_folder):
                print(f"Stock folder {stock} does not exist in target.")
                continue

            for year in year_list:
                year_input_folder = os.path.join(stock_input_folder, year)
                target_file_path = os.path.join(stock_target_folder, f"{stock}_{year}_daily_data.csv")

                if not os.path.exists(year_input_folder):
                    print(f"Year folder {year} for stock {stock} does not exist in input.")
                    continue

                if not os.path.exists(target_file_path):
                    print(f"Target file {target_file_path} does not exist.")
                    continue

                # 讀取目標資料
                try:
                    target_df = pd.read_csv(target_file_path)
                    target_map = {
                        row['date']: {
                            'high': row['high'],
                            'low': row['low'],
                            'close': row['close'],
                            'large_business': row['large_business']
                        } for _, row in target_df.iterrows()
                    }
                except Exception as e:
                    print(f"Error reading target file {target_file_path}: {e}")
                    continue

                # 遍歷一天資料 (30筆) 的 CSV
                for file_name in os.listdir(year_input_folder):
                    if file_name.endswith("_top30_data.csv"):
                        input_file_path = os.path.join(year_input_folder, file_name)

                        try:
                            # 讀取一天的輸入資料
                            df = pd.read_csv(input_file_path)

                            # 檢查資料是否有內容
                            if df.empty or len(df) != 30:
                                print(f"File {input_file_path} has invalid data.")
                                continue

                            # 輸入數據：一天的數據作為輸入 (30筆)
                            input_data = df[['open', 'high', 'low', 'close', 'volume']].values

                            # 匹配目標數據
                            date_key = file_name.split("_")[1]  # 提取年月日
                            if date_key not in target_map:
                                print(f"Target for {date_key} not found in {target_file_path}.")
                                continue

                            target_data = target_map[date_key]
                            target_tensor = torch.tensor(
                                [
                                    target_data['high'],
                                    target_data['low'],
                                    target_data['close'],
                                    target_data['large_business']
                                ],
                                dtype=torch.float32
                            )

                            self.data.append(input_data)
                            self.targets.append(target_tensor)

                        except Exception as e:
                            print(f"Error processing file {input_file_path}: {e}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), self.targets[idx]


In [5]:
# 使用範例
input_folder = "final_input"
target_folder = "final_output"
# 股票清單
stock_list = ["2308"]
# 年份清單
year_list = ["2020", "2021", "2022"]
# 建立 Dataset
stock_dataset = StockDataset(input_folder, target_folder, stock_list, year_list)

# 切分訓練集和驗證集
train_size = int(0.8 * len(stock_dataset))
valid_size = len(stock_dataset) - train_size
train_dataset, valid_dataset = random_split(stock_dataset, [train_size, valid_size])

# 建立 DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# 測試 DataLoader
for batch_idx, (inputs, targets) in enumerate(train_loader):
    print(f"Train Batch {batch_idx} - Inputs: {inputs.shape}, Targets: {targets.shape}")
    break

for batch_idx, (inputs, targets) in enumerate(valid_loader):
    print(f"Validation Batch {batch_idx} - Inputs: {inputs.shape}, Targets: {targets.shape}")
    break


Train Batch 0 - Inputs: torch.Size([32, 30, 5]), Targets: torch.Size([32, 4])
Validation Batch 0 - Inputs: torch.Size([32, 30, 5]), Targets: torch.Size([32, 4])


In [6]:
# # 定義 LSTM 模型 v1 0.6572
# class LSTM(nn.Module):
#     def __init__(self, input_size=5, output_size=4):
#         super(LSTM, self).__init__()
#         self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=128, batch_first=True, bidirectional=True)  # 輸入大小改為 5
#         self.mish = nn.Mish()
#         self.lstm2 = nn.LSTM(input_size=256, hidden_size=128, batch_first=True, bidirectional=True)
#         self.batch_norm1 = nn.LayerNorm(256)  # 處理最後一個時間步[:, -1, :]
#         self.fc1 = nn.Linear(256, 128)
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 16)  # 輸出大小改為 output_size - 1 (前 3 個數值)
#         self.fc4 = nn.Linear(16, output_size - 1)  # 輸出大小改為 output_size - 1 (前 3 個數值)
#         self.fc_sigmoid = nn.Linear(16, 1)  # 單獨處理最後一個數值
#         self.dropout = nn.Dropout(0.5)
#         self.prelu1 = nn.PReLU()
#         self.prelu2 = nn.PReLU()

#         self._initialize_weights()

#     def _initialize_weights(self):
#         for m in self.modules():
#             if isinstance(m, nn.Linear):
#                 nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
#                 if m.bias is not None:
#                     nn.init.constant_(m.bias, 0)
#             elif isinstance(m, nn.LSTM):
#                 for name, param in m.named_parameters():
#                     if 'weight_ih' in name:
#                         nn.init.kaiming_normal_(param.data, nonlinearity='relu')
#                     elif 'weight_hh' in name:
#                         nn.init.orthogonal_(param.data)
#                     elif 'bias' in name:
#                         nn.init.constant_(param.data, 0)
#             elif isinstance(m, nn.BatchNorm1d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)

#     def forward(self, x):
#         x, _ = self.lstm1(x)
#         x = self.mish(x)
#         x, _ = self.lstm2(x)
#         x = self.mish(x)
#         x = torch.mean(x, dim=1)
#         # x = self.batch_norm1(x)  # 處理最後一個時間步[:, -1, :]
#         x = self.fc1(x)
#         x = self.prelu1(x)
#         x = self.fc2(x)
#         x = self.prelu2(x)
#         x = self.fc3(x)
#         if self.training:
#             x = self.dropout(x)      
#         # 分離兩部分輸出
#         out_main = self.fc4(x)  # 前 3 個數值
#         out_sigmoid = torch.sigmoid(self.fc_sigmoid(x))  # 單獨使用 Sigmoid 激活

#         # 合併輸出
#         output = torch.cat((out_main, out_sigmoid), dim=1)
#         return output

In [7]:
# # 定義 LSTM 模型 v2 6840
# class LSTM(nn.Module):
#     def __init__(self, input_size=5, output_size=4):
#         super(LSTM, self).__init__()
#         self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=128, batch_first=True, bidirectional=True)  # 輸入大小改為 5
#         self.mish = nn.Mish()
#         self.lstm2 = nn.LSTM(input_size=256, hidden_size=128, batch_first=True, bidirectional=True)
#         self.batch_norm1 = nn.BatchNorm1d(256)  # 處理最後一個時間步[:, -1, :]
#         self.fc1 = nn.Linear(256, 128)
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 16)  # 輸出大小改為 output_size - 1 (前 3 個數值)
#         self.fc4 = nn.Linear(16, output_size - 1)  # 輸出大小改為 output_size - 1 (前 3 個數值)
#         self.fc_sigmoid = nn.Linear(16, 1)  # 單獨處理最後一個數值
#         self.dropout = nn.Dropout(0.5)

#         self._initialize_weights()

#     def _initialize_weights(self):
#         for m in self.modules():
#             if isinstance(m, nn.Linear):
#                 nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
#                 if m.bias is not None:
#                     nn.init.constant_(m.bias, 0)
#             elif isinstance(m, nn.LSTM):
#                 for name, param in m.named_parameters():
#                     if 'weight_ih' in name:
#                         nn.init.kaiming_normal_(param.data, nonlinearity='relu')
#                     elif 'weight_hh' in name:
#                         nn.init.orthogonal_(param.data)
#                     elif 'bias' in name:
#                         nn.init.constant_(param.data, 0)
#             elif isinstance(m, nn.BatchNorm1d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)

#     def forward(self, x):
#         x, _ = self.lstm1(x)
#         x = self.mish(x)
#         x, _ = self.lstm2(x)
#         x = self.mish(x)
#         x = torch.mean(x, dim=1)
#         # x = self.batch_norm1(x)  # 處理最後一個時間步[:, -1, :]
#         x = self.fc1(x)
#         x = self.mish(x)
#         x = self.fc2(x)
#         x = self.mish(x)
#         x = self.fc3(x)
#         if self.training:
#             x = self.dropout(x)      
#         # 分離兩部分輸出
#         out_main = self.fc4(x)  # 前 3 個數值
#         out_sigmoid = torch.sigmoid(self.fc_sigmoid(x))  # 單獨使用 Sigmoid 激活

#         # 合併輸出
#         output = torch.cat((out_main, out_sigmoid), dim=1)
#         return output

In [8]:
# 定義 LSTM 模型 v3 0.6420
class LSTM(nn.Module):
    def __init__(self, input_size=5, output_size=4):
        super(LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=128, batch_first=True, bidirectional=True)  # 輸入大小改為 5
        self.mish = nn.Mish()
        self.lstm2 = nn.LSTM(input_size=256, hidden_size=128, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(256, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)  # 輸出大小改為 output_size - 1 (前 3 個數值)
        self.fc = nn.Linear(128, 32)  # 輸出大小改為 output_size - 1 (前 3 個數值)
        self.fc4 = nn.Linear(32, output_size - 1)  # 輸出大小改為 output_size - 1 (前 3 個數值)
        self.fc_sigmoid = nn.Linear(32, 1)  # 單獨處理最後一個數值
        self.dropout = nn.Dropout(0.5)
        self.prelu1 = nn.PReLU()
        self.prelu2 = nn.PReLU()
        self.prelu3 = nn.PReLU()    

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.LSTM):
                for name, param in m.named_parameters():
                    if 'weight_ih' in name:
                        nn.init.kaiming_normal_(param.data, nonlinearity='relu')
                    elif 'weight_hh' in name:
                        nn.init.orthogonal_(param.data)
                    elif 'bias' in name:
                        nn.init.constant_(param.data, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.mish(x)
        x, _ = self.lstm2(x)
        x = self.mish(x)
        y = torch.mean(x, dim=1)
        # x = self.batch_norm1(x)  # 處理最後一個時間步[:, -1, :]
        x = self.fc1(y)
        x = self.prelu1(x)
        x = self.fc2(x)
        x = self.prelu2(x)
        x = self.fc3(x+y)
        x = self.prelu3(x)
        x = self.fc(x)
        if self.training:
            x = self.dropout(x)      
        # 分離兩部分輸出
        out_main = self.fc4(x)  # 前 3 個數值
        out_sigmoid = torch.sigmoid(self.fc_sigmoid(x))  # 單獨使用 Sigmoid 激活

        # 合併輸出
        output = torch.cat((out_main, out_sigmoid), dim=1)
        return output

In [9]:
class EarlyStopping:
    def __init__(self, model_save_path,model_name ,only_save = False, patience=7, verbose=False, delta=0):
        """
        :param patience: 没有改善后等待的轮数
        :param verbose: 是否打印消息
        :param delta: 显著改善的最小变化
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
        self.model_save_path = model_save_path
        self.only_save = only_save
        self.lastloss = 0
        self.model_name = model_name

    def __call__(self, val_loss, model,epoch):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model,epoch)
        elif score < self.best_score + self.delta:
          if not self.only_save:
              self.counter += 1
              if self.verbose:
                  print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
              if self.counter >= self.patience:
                  self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model,epoch)
            self.counter = 0

    def save_checkpoint(self, val_loss, model,epoch):
        """
        保存模型
        """
        if self.verbose:
            print(f'Validation loss decreased ({-self.best_score:.8f} --> {self.lastloss:.8f}).  Saving model ...')
        torch.save(model.state_dict(), self.model_save_path+"/"+ self.model_name +".pt")  # 保存模型
        self.lastloss = val_loss


In [10]:
torch.manual_seed(12345)
# 建立模型
model = LSTM(input_size=5, output_size=4).to(device)

loss_mse = nn.MSELoss()
loss_bce = nn.BCELoss ()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

earlystopping = EarlyStopping(model_path,model_name,only_save, patience, verbose)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=10, factor=0.5)

In [11]:
# 添加驗證函數
def Validation(model, valid_loader):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in tqdm(valid_loader, desc="Validation Progress"):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = loss_mse(outputs[:, :-1], targets[:, :-1]) + loss_bce(outputs[:, -1], targets[:, -1])
            val_loss += loss.item()
    return val_loss / len(valid_loader)


In [12]:
for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0
        with tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}") as train_progress:
            for inputs, targets in train_progress:
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = loss_mse(outputs[:, :-1], targets[:, :-1]) + loss_bce(outputs[:, -1], targets[:, -1])
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                train_progress.set_postfix(loss=loss.item())

        val_loss = Validation(model, valid_loader)
        print(f"Epoch {epoch}: Train Loss = {train_loss / len(train_loader):.4f}, Validation Loss = {val_loss:.4f}")
        scheduler.step(val_loss)
        print("lr:",optimizer.param_groups[0]['lr'])
        earlystopping(val_loss, model, epoch)
        if earlystopping.early_stop:
            print("Early stopping")
            break

Validation Progress: 100%|██████████| 5/5 [00:00<00:00,  8.61it/s]


Epoch 1: Train Loss = 1.5370, Validation Loss = 1.2314
lr: 0.0001
Validation loss decreased (1.23136407 --> 0.00000000).  Saving model ...


Validation Progress: 100%|██████████| 5/5 [00:00<00:00, 291.89it/s]


Epoch 2: Train Loss = 1.2458, Validation Loss = 1.0859
lr: 0.0001
Validation loss decreased (1.08588614 --> 1.23136407).  Saving model ...


Validation Progress: 100%|██████████| 5/5 [00:00<00:00, 283.44it/s]


...

Epoch 125: Train Loss = 0.6041, Validation Loss = 0.6499
lr: 3.125e-06
EarlyStopping counter: 48 out of 50


Validation Progress: 100%|██████████| 5/5 [00:00<00:00, 289.92it/s]


Epoch 126: Train Loss = 0.5960, Validation Loss = 0.6480
lr: 3.125e-06
EarlyStopping counter: 49 out of 50


Validation Progress: 100%|██████████| 5/5 [00:00<00:00, 288.98it/s]

Epoch 127: Train Loss = 0.6250, Validation Loss = 0.6485
lr: 3.125e-06
EarlyStopping counter: 50 out of 50
Early stopping
